In [17]:
import pandas as pd
import matplotlib.pyplot as plt

## BTC

In [18]:
btc = pd.read_csv("BTC/btc_final.csv", parse_dates=["timestamp"])

### Moving Average (MA)

In [19]:
btc["ma_6"] = btc["close_price"].rolling(window=6).mean().round(4)
btc["ma_24"] = btc["close_price"].rolling(window=24).mean().round(4)
btc["ma_crossover_signal"] = btc.apply(
    lambda row: (
        1 if row["ma_6"] > row["ma_24"]
        else 0
    ) if pd.notna(row["ma_6"]) and pd.notna(row["ma_24"]) else None,
    axis=1
)    # 1: short MA is above long MA (buy signal), 0: short MA is below long MA (no buy signal)
btc["ma_crossover_signal"] = btc["ma_crossover_signal"].astype("Int64")
btc["ma_crossover_description"] = btc["ma_crossover_signal"].apply(
    lambda x: (
        "short MA (6h) is above long MA (24h) → buy signal"
        if x == 1 else
        "short MA (6h) is below long MA (24h) → no buy signal"
        if x == 0 else 
        None
    ) if pd.notna(x) else None
)
btc.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_btc,avg_fee_btc,total_size_used_mb,unique_active_wallets,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description
0,2024-05-01 00:00:00,60208.8170,8511.2535,5.141303e+08,27131,41488.3624,0.000064,11.7035,50034,1.192806e+12,NaN,NaN,<NA>,None
1,2024-05-01 01:00:00,60095.5002,8853.2447,5.321780e+08,25263,29720.9307,0.000063,11.4590,39480,1.185496e+12,NaN,NaN,<NA>,None
2,2024-05-01 02:00:00,59886.5202,8690.0466,5.193491e+08,21228,29104.3936,0.000060,7.8200,37710,1.183924e+12,NaN,NaN,<NA>,None
3,2024-05-01 03:00:00,60206.0485,5206.5224,3.127513e+08,19018,62407.2166,0.000070,8.3795,42124,1.179127e+12,NaN,NaN,<NA>,None
4,2024-05-01 04:00:00,60241.4675,4977.9718,2.998299e+08,26684,44802.0762,0.000057,11.0749,36411,1.185334e+12,NaN,NaN,<NA>,None
5,2024-05-01 05:00:00,60005.2504,5928.1266,3.560629e+08,11898,26359.2250,0.000092,4.5227,27976,1.185417e+12,60107.2673,NaN,<NA>,None
6,2024-05-01 06:00:00,59501.2836,7846.8091,4.688953e+08,17902,27977.1218,0.000082,7.7468,42564,1.180281e+12,59989.3451,NaN,<NA>,None
7,2024-05-01 07:00:00,57466.2288,48781.7905,2.831902e+09,18476,80240.6812,0.000082,7.7279,46975,1.174252e+12,59551.1332,NaN,<NA>,None
8,2024-05-01 08:00:00,57103.0541,34919.7273,1.990814e+09,36958,77431.8478,0.000062,14.1263,59083,1.129262e+12,59087.2222,NaN,<NA>,None
9,2024-05-01 09:00:00,57096.7474,14509.3452,8.308224e+08,36684,38395.1546,0.000054,13.1422,48869,1.119474e+12,58569.0053,NaN,<NA>,None


### MACD (Moving Average Convergence Divergence)

`MACD`: a momentum indicator that shows the relationship between two Exponential Moving Averages (EMA)

`EMA`: gives more weight to recent prices, making it more responsive than a simple moving average (MA)

MACD consists of 3 main components:
1. MACD Line = EMA(12) - EMA(26): measures trend momentum.
2. Signal Line = EMA(9) of MACD: smooths the MACD line.
3. MACD Histogram = MACD Line - Signal Line: shows the strength of momentum change.

- A crossover (MACD > Signal) is commonly interpreted as a buy signal.
- A crossover (MACD < Signal) is commonly interpreted as a sell signal.

- 当 MACD线 > Signal线：表示上涨动量增强，通常是买入点
- 当 MACD线 < Signal线：表示下跌动量增强，通常是卖出点
- 两者的差值越大，说明动量越强


In [20]:
# 1. Compute short-term EMA (12h)
btc["ema_12"] = btc['close_price'].ewm(span=12, adjust=False).mean().round(4)

# 2. Compute long-term EMA (26h)
btc["ema_26"] = btc['close_price'].ewm(span=26, adjust=False).mean().round(4)

# 3. MACD line = ema_12 - ema_26
btc["macd"] = (btc["ema_12"] - btc["ema_26"]).round(4)

# 4. Signal line = EMA(9) of MACD3
    # 对 MACD 再做一次 EMA（指数移动平均），它是一个“更平滑的 MACD 曲线”，用于生成买卖信号。
btc["macd_signal"] = btc["macd"].ewm(span=9, adjust=False).mean().round(4)

# 5. Compute MACD histogram (MACD - Signal)
btc["macd_hist"] = (btc["macd"] - btc["macd_signal"]).round(4)

# 6. Generate a crossover signal (1 -> MACD above signal -> buy momentum)
btc["macd_crossover_signal"] = (btc["macd"] > btc["macd_signal"]).astype(int)

# 7. Convert to MACD crossover description
btc["macd_crossover_description"] = btc["macd_crossover_signal"].apply(
    lambda x: (
        "MACD (12h vs 26h EMA) is above signal line → bullish momentum"
        if x == 1 else
        "MACD (12h vs 26h EMA) is below signal line → no bullish signal"
        if x == 0 else None
    )
)

In [21]:
btc.head()

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_btc,avg_fee_btc,total_size_used_mb,unique_active_wallets,market_cap,...,ma_24,ma_crossover_signal,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description
0,2024-05-01 00:00:00,60208.8170,8511.2535,5.141303e+08,27131,41488.3624,0.000064,11.7035,50034,1.192806e+12,...,NaN,<NA>,None,60208.8170,60208.8170,0.0000,0.0000,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...
1,2024-05-01 01:00:00,60095.5002,8853.2447,5.321780e+08,25263,29720.9307,0.000063,11.4590,39480,1.185496e+12,...,NaN,<NA>,None,60191.3836,60200.4232,-9.0396,-1.8079,-7.2317,0,MACD (12h vs 26h EMA) is below signal line → n...
2,2024-05-01 02:00:00,59886.5202,8690.0466,5.193491e+08,21228,29104.3936,0.000060,7.8200,37710,1.183924e+12,...,NaN,<NA>,None,60144.4816,60177.1711,-32.6895,-7.9842,-24.7053,0,MACD (12h vs 26h EMA) is below signal line → n...
3,2024-05-01 03:00:00,60206.0485,5206.5224,3.127513e+08,19018,62407.2166,0.000070,8.3795,42124,1.179127e+12,...,NaN,<NA>,None,60153.9534,60179.3102,-25.3568,-11.4587,-13.8981,0,MACD (12h vs 26h EMA) is below signal line → n...
4,2024-05-01 04:00:00,60241.4675,4977.9718,2.998299e+08,26684,44802.0762,0.000057,11.0749,36411,1.185334e+12,...,NaN,<NA>,None,60167.4171,60183.9144,-16.4973,-12.4665,-4.0308,0,MACD (12h vs 26h EMA) is below signal line → n...


### Transaction Count Ratio

Compares the current hour's transaction count to the average of the past 24 hours.

It helps assess whether on-chain activity is surging, normal, or low.

In [22]:
# 1. Calculate 24h rolling average of transaction count
btc["txn_count_mean_24h"] = btc["transaction_count"].rolling(window=24).mean()

# 2. Compute the ratio: current transaction count / 24h rolling average
btc["txn_ratio"] = (btc["transaction_count"] / btc["txn_count_mean_24h"]).round(4)
btc.head(30)


,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_btc,avg_fee_btc,total_size_used_mb,unique_active_wallets,market_cap,...,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description,txn_count_mean_24h,txn_ratio
0,2024-05-01 00:00:00,60208.8170,8511.2535,5.141303e+08,27131,41488.3624,0.000064,11.7035,50034,1.192806e+12,...,None,60208.8170,60208.8170,0.0000,0.0000,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
1,2024-05-01 01:00:00,60095.5002,8853.2447,5.321780e+08,25263,29720.9307,0.000063,11.4590,39480,1.185496e+12,...,None,60191.3836,60200.4232,-9.0396,-1.8079,-7.2317,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
2,2024-05-01 02:00:00,59886.5202,8690.0466,5.193491e+08,21228,29104.3936,0.000060,7.8200,37710,1.183924e+12,...,None,60144.4816,60177.1711,-32.6895,-7.9842,-24.7053,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
3,2024-05-01 03:00:00,60206.0485,5206.5224,3.127513e+08,19018,62407.2166,0.000070,8.3795,42124,1.179127e+12,...,None,60153.9534,60179.3102,-25.3568,-11.4587,-13.8981,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
4,2024-05-01 04:00:00,60241.4675,4977.9718,2.998299e+08,26684,44802.0762,0.000057,11.0749,36411,1.185334e+12,...,None,60167.4171,60183.9144,-16.4973,-12.4665,-4.0308,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
5,2024-05-01 05:00:00,60005.2504,5928.1266,3.560629e+08,11898,26359.2250,0.000092,4.5227,27976,1.185417e+12,...,None,60142.4684,60170.6800,-28.2116,-15.6155,-12.5961,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
6,2024-05-01 06:00:00,59501.2836,7846.8091,4.688953e+08,17902,27977.1218,0.000082,7.7468,42564,1.180281e+12,...,None,60043.8246,60121.0951,-77.2705,-27.9465,-49.3240,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
7,2024-05-01 07:00:00,57466.2288,48781.7905,2.831902e+09,18476,80240.6812,0.000082,7.7279,46975,1.174252e+12,...,None,59647.2714,59924.4384,-277.1670,-77.7906,-199.3764,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
8,2024-05-01 08:00:00,57103.0541,34919.7273,1.990814e+09,36958,77431.8478,0.000062,14.1263,59083,1.129262e+12,...,None,59255.8533,59715.4469,-459.5936,-154.1512,-305.4424,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN
9,2024-05-01 09:00:00,57096.7474,14509.3452,8.308224e+08,36684,38395.1546,0.000054,13.1422,48869,1.119474e+12,...,None,58923.6832,59521.4692,-597.7860,-242.8782,-354.9078,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,NaN


In [23]:
btc["txn_ratio_description"] = btc["txn_ratio"].apply(
    lambda x: (
        "activity surge" if x > 1.2 else
        "low activity" if x < 0.8 else
        "normal activity"
    ) if pd.notna(x) else None
)

btc.drop(columns="txn_count_mean_24h", inplace=True)

In [24]:
btc["txn_ratio_description"].value_counts()

txn_ratio_description
low activity       3029
normal activity    2592
activity surge     2396
Name: count, dtype: int64

### Bollinger Bands

Bollinger Bands consist of three lines:
- Middle Band: Simple Moving Average (usually 20-period)
- Upper Band: Middle + 2 × standard deviation
- Lower Band: Middle - 2 × standard deviation

These bands help identify whether the price is relatively high or low
compared to recent history, and are useful for detecting breakout or sideways markets.

In [25]:
# 1. Calculate 20-period moving average (middle band)
btc["bb_mid"] = btc["close_price"].rolling(window=20).mean()

# 2. Calculate 20-period rolling standard deviation
btc["bb_std"] = btc["close_price"].rolling(window=20).std()

# 3. Compute upper and lower bands
btc["bb_upper"] = (btc["bb_mid"] + 2 * btc["bb_std"]).round(4)
btc["bb_lower"] = (btc["bb_mid"] - 2 * btc["bb_std"]).round(4)

# 4. Bollinger band width
btc["bb_position"] = btc.apply(
    lambda row: (
        "above upper band (possible breakout)" if row["close_price"] > row["bb_upper"]
        else "below lower band (possible breakdown)" if row["close_price"] < row["bb_lower"]
        else "within band (normal range)"
    ) if pd.notna(row["bb_upper"]) and pd.notna(row["bb_lower"]) else None, 
    axis=1
)

This field classifies the current close price relative to the Bollinger Bands.

- If the price is above the upper band → it's called a **breakout**.
    - Meaning: price has moved beyond normal volatility range on the upside.
    - Traders often interpret this as a strong bullish momentum or start of an upward trend.

- If the price is below the lower band → it's called a **rebound**.
    - Meaning: price may have dropped too sharply and is likely to bounce back.
    - Traders interpret this as a potential oversold signal or reversal opportunity.

- If the price is between upper and lower bands → it's in the **normal range** (sideways).
    - Meaning: market is moving within expected volatility bounds.

In [26]:
btc.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_btc,avg_fee_btc,total_size_used_mb,unique_active_wallets,market_cap,...,macd_hist,macd_crossover_signal,macd_crossover_description,txn_ratio,txn_ratio_description,bb_mid,bb_std,bb_upper,bb_lower,bb_position
0,2024-05-01 00:00:00,60208.8170,8511.2535,5.141303e+08,27131,41488.3624,0.000064,11.7035,50034,1.192806e+12,...,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
1,2024-05-01 01:00:00,60095.5002,8853.2447,5.321780e+08,25263,29720.9307,0.000063,11.4590,39480,1.185496e+12,...,-7.2317,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
2,2024-05-01 02:00:00,59886.5202,8690.0466,5.193491e+08,21228,29104.3936,0.000060,7.8200,37710,1.183924e+12,...,-24.7053,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
3,2024-05-01 03:00:00,60206.0485,5206.5224,3.127513e+08,19018,62407.2166,0.000070,8.3795,42124,1.179127e+12,...,-13.8981,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
4,2024-05-01 04:00:00,60241.4675,4977.9718,2.998299e+08,26684,44802.0762,0.000057,11.0749,36411,1.185334e+12,...,-4.0308,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
5,2024-05-01 05:00:00,60005.2504,5928.1266,3.560629e+08,11898,26359.2250,0.000092,4.5227,27976,1.185417e+12,...,-12.5961,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
6,2024-05-01 06:00:00,59501.2836,7846.8091,4.688953e+08,17902,27977.1218,0.000082,7.7468,42564,1.180281e+12,...,-49.3240,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
7,2024-05-01 07:00:00,57466.2288,48781.7905,2.831902e+09,18476,80240.6812,0.000082,7.7279,46975,1.174252e+12,...,-199.3764,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
8,2024-05-01 08:00:00,57103.0541,34919.7273,1.990814e+09,36958,77431.8478,0.000062,14.1263,59083,1.129262e+12,...,-305.4424,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
9,2024-05-01 09:00:00,57096.7474,14509.3452,8.308224e+08,36684,38395.1546,0.000054,13.1422,48869,1.119474e+12,...,-354.9078,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None


### Price Change Percentage

This indicator measures the relative change in closing price compared to the previous time step.

It's useful for identifying short-term price momentum (up/down/flat).

Formula:

   $\frac{\text{close}_t - \text{close}_{t-1}}{\text{close}_{t-1}}$

In [27]:
btc["price_change_pct"] = btc["close_price"].pct_change().round(6)
btc.head(10)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_btc,avg_fee_btc,total_size_used_mb,unique_active_wallets,market_cap,...,macd_crossover_signal,macd_crossover_description,txn_ratio,txn_ratio_description,bb_mid,bb_std,bb_upper,bb_lower,bb_position,price_change_pct
0,2024-05-01 00:00:00,60208.8170,8511.2535,5.141303e+08,27131,41488.3624,0.000064,11.7035,50034,1.192806e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,NaN
1,2024-05-01 01:00:00,60095.5002,8853.2447,5.321780e+08,25263,29720.9307,0.000063,11.4590,39480,1.185496e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.001882
2,2024-05-01 02:00:00,59886.5202,8690.0466,5.193491e+08,21228,29104.3936,0.000060,7.8200,37710,1.183924e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.003477
3,2024-05-01 03:00:00,60206.0485,5206.5224,3.127513e+08,19018,62407.2166,0.000070,8.3795,42124,1.179127e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,0.005336
4,2024-05-01 04:00:00,60241.4675,4977.9718,2.998299e+08,26684,44802.0762,0.000057,11.0749,36411,1.185334e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,0.000588
5,2024-05-01 05:00:00,60005.2504,5928.1266,3.560629e+08,11898,26359.2250,0.000092,4.5227,27976,1.185417e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.003921
6,2024-05-01 06:00:00,59501.2836,7846.8091,4.688953e+08,17902,27977.1218,0.000082,7.7468,42564,1.180281e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.008399
7,2024-05-01 07:00:00,57466.2288,48781.7905,2.831902e+09,18476,80240.6812,0.000082,7.7279,46975,1.174252e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.034202
8,2024-05-01 08:00:00,57103.0541,34919.7273,1.990814e+09,36958,77431.8478,0.000062,14.1263,59083,1.129262e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.006320
9,2024-05-01 09:00:00,57096.7474,14509.3452,8.308224e+08,36684,38395.1546,0.000054,13.1422,48869,1.119474e+12,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.000110


In [28]:
# Classify movement for LLM prompt
btc["price_movement"] = btc["price_change_pct"].apply(
    lambda x: (
        "significant increase" if x > 0.01 else
        "slight increase" if x > 0.002 else
        "no significant change" if x > - 0.002 else
        "slight decrease" if x > -0.01 else
        "significant decrease"
    ) if pd.notna(x) else None 
)

In [29]:
btc["price_movement"].value_counts()

price_movement
no significant change    3551
slight increase          2074
slight decrease          1891
significant decrease      273
significant increase      250
Name: count, dtype: int64

In [30]:
btc.to_csv("BTC/btc_full.csv", index=False)

## ETH

In [31]:
eth = pd.read_csv("ETH/eth_final.csv", parse_dates=["timestamp"])
eth

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_eth,unique_active_wallets,avg_gas_price_gwei,total_gas_used,market_cap
0,2024-05-01 00:00:00,2997.2957,87041.9491,2.613718e+08,50831,53248.2181,25482,8.2190,4542529445,3.687864e+11
1,2024-05-01 01:00:00,3000.8196,90117.9517,2.701605e+08,49831,75066.1618,26422,8.1389,4578797065,3.659825e+11
2,2024-05-01 02:00:00,2985.5093,92728.8428,2.763142e+08,48229,29038.2851,25233,9.2955,4518099410,3.665995e+11
3,2024-05-01 03:00:00,3001.9196,45859.1077,1.372769e+08,48785,27267.6164,27274,8.1441,4530654967,3.641239e+11
4,2024-05-01 04:00:00,3003.8646,41604.4323,1.250516e+08,47804,35669.8495,26537,7.1900,4541463899,3.655860e+11
...,...,...,...,...,...,...,...,...,...,...
8035,2025-03-31 19:00:00,1827.5233,170162.2498,3.123563e+08,62628,67909.5841,28120,2.1402,5401486521,2.218443e+11
8036,2025-03-31 20:00:00,1819.8772,106165.0559,1.941037e+08,62727,81357.4728,26322,3.3118,5495013163,2.203444e+11
8037,2025-03-31 21:00:00,1824.6029,72938.0947,1.331678e+08,60498,106047.8367,25494,2.9551,5359587938,2.197182e+11
8038,2025-03-31 22:00:00,1824.7112,72546.7354,1.321121e+08,59763,40238.3265,29035,1.8836,5432212218,2.201118e+11


### Moving Average (MA)

In [32]:
eth["ma_6"] = eth["close_price"].rolling(window=6).mean().round(4)
eth["ma_24"] = eth["close_price"].rolling(window=24).mean().round(4)
eth["ma_crossover_signal"] = eth.apply(
    lambda row: (
        1 if row["ma_6"] > row["ma_24"]
        else 0
    ) if pd.notna(row["ma_6"]) and pd.notna(row["ma_24"]) else None,
    axis=1
).astype("Int64")    # 1: short MA is above long MA (buy signal), 0: short MA is below long MA (no buy signal)
eth["ma_crossover_description"] = eth["ma_crossover_signal"].apply(
    lambda x: (
        "short MA (6h) is above long MA (24h) → buy signal"
        if x == 1 else
        "short MA (6h) is below long MA (24h) → no buy signal"
        if x == 0 else 
        None
    ) if pd.notna(x) else None
)
eth.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_eth,unique_active_wallets,avg_gas_price_gwei,total_gas_used,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description
0,2024-05-01 00:00:00,2997.2957,87041.9491,2.613718e+08,50831,53248.2181,25482,8.2190,4542529445,3.687864e+11,NaN,NaN,<NA>,None
1,2024-05-01 01:00:00,3000.8196,90117.9517,2.701605e+08,49831,75066.1618,26422,8.1389,4578797065,3.659825e+11,NaN,NaN,<NA>,None
2,2024-05-01 02:00:00,2985.5093,92728.8428,2.763142e+08,48229,29038.2851,25233,9.2955,4518099410,3.665995e+11,NaN,NaN,<NA>,None
3,2024-05-01 03:00:00,3001.9196,45859.1077,1.372769e+08,48785,27267.6164,27274,8.1441,4530654967,3.641239e+11,NaN,NaN,<NA>,None
4,2024-05-01 04:00:00,3003.8646,41604.4323,1.250516e+08,47804,35669.8495,26537,7.1900,4541463899,3.655860e+11,NaN,NaN,<NA>,None
5,2024-05-01 05:00:00,2992.2338,61148.8662,1.830785e+08,46818,23924.5542,25218,7.4799,4527735659,3.665604e+11,2996.9404,NaN,<NA>,None
6,2024-05-01 06:00:00,2952.8314,93689.7612,2.783242e+08,49805,23264.6681,26463,7.6935,4553312046,3.642597e+11,2989.5297,NaN,<NA>,None
7,2024-05-01 07:00:00,2867.5894,368547.2730,1.066140e+09,49596,83393.7018,22182,22.0712,4633450219,3.611731e+11,2967.3247,NaN,<NA>,None
8,2024-05-01 08:00:00,2859.6975,271661.9017,7.733636e+08,49251,74381.2800,22358,20.4277,4511887791,3.495610e+11,2946.3560,NaN,<NA>,None
9,2024-05-01 09:00:00,2874.1493,127704.8928,3.673134e+08,49242,92524.2244,22534,13.9404,4459155419,3.474973e+11,2925.0610,NaN,<NA>,None


### MACD (Moving Average Convergence Divergence)

`MACD`: a momentum indicator that shows the relationship between two Exponential Moving Averages (EMA)

`EMA`: gives more weight to recent prices, making it more responsive than a simple moving average (MA)

MACD consists of 3 main components:
1. MACD Line = EMA(12) - EMA(26): measures trend momentum.
2. Signal Line = EMA(9) of MACD: smooths the MACD line.
3. MACD Histogram = MACD Line - Signal Line: shows the strength of momentum change.

- A crossover (MACD > Signal) is commonly interpreted as a buy signal.
- A crossover (MACD < Signal) is commonly interpreted as a sell signal.

- 当 MACD线 > Signal线：表示上涨动量增强，通常是买入点
- 当 MACD线 < Signal线：表示下跌动量增强，通常是卖出点
- 两者的差值越大，说明动量越强


In [33]:
# 1. Compute short-term EMA (12h)
eth["ema_12"] = eth['close_price'].ewm(span=12, adjust=False).mean().round(4)

# 2. Compute long-term EMA (26h)
eth["ema_26"] = eth['close_price'].ewm(span=26, adjust=False).mean().round(4)

# 3. MACD line = ema_12 - ema_26
eth["macd"] = (eth["ema_12"] - eth["ema_26"]).round(4)

# 4. Signal line = EMA(9) of MACD3
    # 对 MACD 再做一次EMA（指数移动平均），它是一个“更平滑的 MACD 曲线”，用于生成买卖信号。
eth["macd_signal"] = eth["macd"].ewm(span=9, adjust=False).mean().round(4)

# 5. Compute MACD histogram (MACD - Signal)
eth["macd_hist"] = (eth["macd"] - eth["macd_signal"]).round(4)

# 6. Generate a crossover signal (1 -> MACD above signal -> buy momentum)
eth["macd_crossover_signal"] = (eth["macd"] > eth["macd_signal"]).astype(int)

# 7. Convert to MACD crossover description
eth["macd_crossover_description"] = eth["macd_crossover_signal"].apply(
    lambda x: (
        "MACD (12h vs 26h EMA) is above signal line → bullish momentum"
        if x == 1 else
        "MACD (12h vs 26h EMA) is below signal line → no bullish signal"
        if x == 0 else None
    )
)

In [34]:
eth.head()

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_eth,unique_active_wallets,avg_gas_price_gwei,total_gas_used,market_cap,...,ma_24,ma_crossover_signal,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description
0,2024-05-01 00:00:00,2997.2957,87041.9491,2.613718e+08,50831,53248.2181,25482,8.2190,4542529445,3.687864e+11,...,NaN,<NA>,None,2997.2957,2997.2957,0.0000,0.0000,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...
1,2024-05-01 01:00:00,3000.8196,90117.9517,2.701605e+08,49831,75066.1618,26422,8.1389,4578797065,3.659825e+11,...,NaN,<NA>,None,2997.8378,2997.5567,0.2811,0.0562,0.2249,1,MACD (12h vs 26h EMA) is above signal line → b...
2,2024-05-01 02:00:00,2985.5093,92728.8428,2.763142e+08,48229,29038.2851,25233,9.2955,4518099410,3.665995e+11,...,NaN,<NA>,None,2995.9411,2996.6643,-0.7232,-0.0997,-0.6235,0,MACD (12h vs 26h EMA) is below signal line → n...
3,2024-05-01 03:00:00,3001.9196,45859.1077,1.372769e+08,48785,27267.6164,27274,8.1441,4530654967,3.641239e+11,...,NaN,<NA>,None,2996.8609,2997.0536,-0.1927,-0.1183,-0.0744,0,MACD (12h vs 26h EMA) is below signal line → n...
4,2024-05-01 04:00:00,3003.8646,41604.4323,1.250516e+08,47804,35669.8495,26537,7.1900,4541463899,3.655860e+11,...,NaN,<NA>,None,2997.9384,2997.5581,0.3803,-0.0186,0.3989,1,MACD (12h vs 26h EMA) is above signal line → b...


### Transaction Count Ratio

Compares the current hour's transaction count to the average of the past 24 hours.

It helps assess whether on-chain activity is surging, normal, or low.

In [35]:
# 1. Calculate 24h rolling average of transaction count
eth["txn_count_mean_24h"] = eth["transaction_count"].rolling(window=24).mean()

# 2. Compute the ratio: current transaction count / 24h rolling average
eth["txn_ratio"] = (eth["transaction_count"] / eth["txn_count_mean_24h"]).round(4)

# 3. Convert to transaction ratio description
eth["txn_ratio_description"] = eth["txn_ratio"].apply(
    lambda x: (
        "activity surge" if x > 1.1 else
        "low activity" if x < 0.9 else
        "normal activity"
    ) if pd.notna(x) else None
)

eth.drop(columns="txn_count_mean_24h", inplace=True)

In [36]:
eth["txn_ratio_description"].value_counts()

txn_ratio_description
normal activity    6382
low activity        884
activity surge      751
Name: count, dtype: int64

### Bollinger Bands

Bollinger Bands consist of three lines:
- Middle Band: Simple Moving Average (usually 20-period)
- Upper Band: Middle + 2 × standard deviation
- Lower Band: Middle - 2 × standard deviation

These bands help identify whether the price is relatively high or low
compared to recent history, and are useful for detecting breakout or sideways markets.

In [37]:
# 1. Calculate 20-period moving average (middle band)
eth["bb_mid"] = eth["close_price"].rolling(window=20).mean()

# 2. Calculate 20-period rolling standard deviation
eth["bb_std"] = eth["close_price"].rolling(window=20).std()

# 3. Compute upper and lower bands
eth["bb_upper"] = (eth["bb_mid"] + 2 * eth["bb_std"]).round(4)
eth["bb_lower"] = (eth["bb_mid"] - 2 * eth["bb_std"]).round(4)

# 4. Bollinger band width
eth["bb_position"] = eth.apply(
    lambda row: (
        "above upper band (possible breakout)" if row["close_price"] > row["bb_upper"]
        else "below lower band (possible breakdown)" if row["close_price"] < row["bb_lower"]
        else "within band (normal range)"
    ) if pd.notna(row["bb_upper"]) and pd.notna(row["bb_lower"]) else None, 
    axis=1
)

This field classifies the current close price relative to the Bollinger Bands.

- If the price is above the upper band → it's called a **breakout**.
    - Meaning: price has moved beyond normal volatility range on the upside.
    - Traders often interpret this as a strong bullish momentum or start of an upward trend.

- If the price is below the lower band → it's called a **rebound**.
    - Meaning: price may have dropped too sharply and is likely to bounce back.
    - Traders interpret this as a potential oversold signal or reversal opportunity.

- If the price is between upper and lower bands → it's in the **normal range** (sideways).
    - Meaning: market is moving within expected volatility bounds.

In [38]:
eth.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_eth,unique_active_wallets,avg_gas_price_gwei,total_gas_used,market_cap,...,macd_hist,macd_crossover_signal,macd_crossover_description,txn_ratio,txn_ratio_description,bb_mid,bb_std,bb_upper,bb_lower,bb_position
0,2024-05-01 00:00:00,2997.2957,87041.9491,2.613718e+08,50831,53248.2181,25482,8.2190,4542529445,3.687864e+11,...,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
1,2024-05-01 01:00:00,3000.8196,90117.9517,2.701605e+08,49831,75066.1618,26422,8.1389,4578797065,3.659825e+11,...,0.2249,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None
2,2024-05-01 02:00:00,2985.5093,92728.8428,2.763142e+08,48229,29038.2851,25233,9.2955,4518099410,3.665995e+11,...,-0.6235,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
3,2024-05-01 03:00:00,3001.9196,45859.1077,1.372769e+08,48785,27267.6164,27274,8.1441,4530654967,3.641239e+11,...,-0.0744,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
4,2024-05-01 04:00:00,3003.8646,41604.4323,1.250516e+08,47804,35669.8495,26537,7.1900,4541463899,3.655860e+11,...,0.3989,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None
5,2024-05-01 05:00:00,2992.2338,61148.8662,1.830785e+08,46818,23924.5542,25218,7.4799,4527735659,3.665604e+11,...,-0.0675,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
6,2024-05-01 06:00:00,2952.8314,93689.7612,2.783242e+08,49805,23264.6681,26463,7.6935,4553312046,3.642597e+11,...,-2.8706,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
7,2024-05-01 07:00:00,2867.5894,368547.2730,1.066140e+09,49596,83393.7018,22182,22.0712,4633450219,3.611731e+11,...,-9.9100,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
8,2024-05-01 08:00:00,2859.6975,271661.9017,7.733636e+08,49251,74381.2800,22358,20.4277,4511887791,3.495610e+11,...,-14.2769,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
9,2024-05-01 09:00:00,2874.1493,127704.8928,3.673134e+08,49242,92524.2244,22534,13.9404,4459155419,3.474973e+11,...,-15.2814,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None


### Price Change Percentage

This indicator measures the relative change in closing price compared to the previous time step.

It's useful for identifying short-term price momentum (up/down/flat).

Formula:

   $\frac{\text{close}_t - \text{close}_{t-1}}{\text{close}_{t-1}}$

In [39]:
eth["price_change_pct"] = eth["close_price"].pct_change().round(6)

# Classify movement for LLM prompt
eth["price_movement"] = eth["price_change_pct"].apply(
    lambda x: (
        "significant increase" if x > 0.01 else
        "slight increase" if x > 0.002 else
        "no significant change" if x > - 0.002 else
        "slight decrease" if x > -0.01 else
        "significant decrease"
    ) if pd.notna(x) else None 
)
eth["price_movement"].value_counts()

price_movement
no significant change    2816
slight increase          2240
slight decrease          2116
significant decrease      465
significant increase      402
Name: count, dtype: int64

In [40]:
eth.to_csv("ETH/eth_full.csv", index=False)

## SOL

In [41]:
sol = pd.read_csv("SOL/sol_final.csv", parse_dates=["timestamp"])

### Moving Average (MA)

In [42]:
sol["ma_6"] = sol["close_price"].rolling(window=6).mean().round(4)
sol["ma_24"] = sol["close_price"].rolling(window=24).mean().round(4)
sol["ma_crossover_signal"] = sol.apply(
    lambda row: (
        1 if row["ma_6"] > row["ma_24"]
        else 0
    ) if pd.notna(row["ma_6"]) and pd.notna(row["ma_24"]) else None,
    axis=1
).astype("Int64")    # 1: short MA is above long MA (buy signal), 0: short MA is below long MA (no buy signal)
sol["ma_crossover_description"] = sol["ma_crossover_signal"].apply(
    lambda x: (
        "short MA (6h) is above long MA (24h) → buy signal"
        if x == 1 else
        "short MA (6h) is below long MA (24h) → no buy signal"
        if x == 0 else 
        None
    ) if pd.notna(x) else None
)
sol.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,successful_transaction_count,total_fee_sol,avg_fee_sol,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description
0,2024-05-01 00:00:00,125.1005,5.139342e+05,6.457594e+07,8837436,6777203,228.6608,0.000026,5.706838e+10,NaN,NaN,<NA>,None
1,2024-05-01 01:00:00,126.8711,6.834540e+05,8.616440e+07,9286854,7312422,206.7113,0.000022,5.602258e+10,NaN,NaN,<NA>,None
2,2024-05-01 02:00:00,125.0548,7.058468e+05,8.831848e+07,8450646,6534633,217.1579,0.000026,5.677855e+10,NaN,NaN,<NA>,None
3,2024-05-01 03:00:00,126.1589,4.224065e+05,5.295424e+07,9489185,7529496,221.0711,0.000023,5.601521e+10,NaN,NaN,<NA>,None
4,2024-05-01 04:00:00,125.9056,3.028096e+05,3.819878e+07,9238242,7418829,222.2989,0.000024,5.651554e+10,NaN,NaN,<NA>,None
5,2024-05-01 05:00:00,125.0820,3.206541e+05,4.016831e+07,9074138,7142826,171.0434,0.000019,5.628717e+10,125.6955,NaN,<NA>,None
6,2024-05-01 06:00:00,122.8263,6.899530e+05,8.534381e+07,8511218,6724752,162.6879,0.000019,5.588340e+10,125.3164,NaN,<NA>,None
7,2024-05-01 07:00:00,120.5031,1.757072e+06,2.122383e+08,8462130,6879615,202.1013,0.000024,5.527038e+10,124.2551,NaN,<NA>,None
8,2024-05-01 08:00:00,121.0997,1.108013e+06,1.329245e+08,8489463,6847295,182.6292,0.000022,5.386487e+10,123.5959,NaN,<NA>,None
9,2024-05-01 09:00:00,120.7857,7.428299e+05,9.014519e+07,8777861,7057795,179.6191,0.000020,5.375512e+10,122.7004,NaN,<NA>,None


### MACD (Moving Average Convergence Divergence)

`MACD`: a momentum indicator that shows the relationship between two Exponential Moving Averages (EMA)

`EMA`: gives more weight to recent prices, making it more responsive than a simple moving average (MA)

MACD consists of 3 main components:
1. MACD Line = EMA(12) - EMA(26): measures trend momentum.
2. Signal Line = EMA(9) of MACD: smooths the MACD line.
3. MACD Histogram = MACD Line - Signal Line: shows the strength of momentum change.

- A crossover (MACD > Signal) is commonly interpreted as a buy signal.
- A crossover (MACD < Signal) is commonly interpreted as a sell signal.

- 当 MACD线 > Signal线：表示上涨动量增强，通常是买入点
- 当 MACD线 < Signal线：表示下跌动量增强，通常是卖出点
- 两者的差值越大，说明动量越强


In [43]:
# 1. Compute short-term EMA (12h)
sol["ema_12"] = sol['close_price'].ewm(span=12, adjust=False).mean().round(4)

# 2. Compute long-term EMA (26h)
sol["ema_26"] = sol['close_price'].ewm(span=26, adjust=False).mean().round(4)

# 3. MACD line = ema_12 - ema_26
sol["macd"] = (sol["ema_12"] - sol["ema_26"]).round(4)

# 4. Signal line = EMA(9) of MACD3
    # 对 MACD 再做一次EMA（指数移动平均），它是一个“更平滑的 MACD 曲线”，用于生成买卖信号。
sol["macd_signal"] = sol["macd"].ewm(span=9, adjust=False).mean().round(4)

# 5. Compute MACD histogram (MACD - Signal)
sol["macd_hist"] = (sol["macd"] - sol["macd_signal"]).round(4)

# 6. Generate a crossover signal (1 -> MACD above signal -> buy momentum)
sol["macd_crossover_signal"] = (sol["macd"] > sol["macd_signal"]).astype(int)

# 7. Convert to MACD crossover description
sol["macd_crossover_description"] = sol["macd_crossover_signal"].apply(
    lambda x: (
        "MACD (12h vs 26h EMA) is above signal line → bullish momentum"
        if x == 1 else
        "MACD (12h vs 26h EMA) is below signal line → no bullish signal"
        if x == 0 else None
    )
)

In [44]:
sol.head()

,timestamp,close_price,volume_base,volume_quote,transaction_count,successful_transaction_count,total_fee_sol,avg_fee_sol,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description
0,2024-05-01 00:00:00,125.1005,513934.1844,64575943.45,8837436,6777203,228.6608,0.000026,5.706838e+10,NaN,NaN,<NA>,None,125.1005,125.1005,0.0000,0.0000,0.0000,0,MACD (12h vs 26h EMA) is below signal line → n...
1,2024-05-01 01:00:00,126.8711,683454.0389,86164402.38,9286854,7312422,206.7113,0.000022,5.602258e+10,NaN,NaN,<NA>,None,125.3729,125.2317,0.1412,0.0282,0.1130,1,MACD (12h vs 26h EMA) is above signal line → b...
2,2024-05-01 02:00:00,125.0548,705846.8439,88318479.60,8450646,6534633,217.1579,0.000026,5.677855e+10,NaN,NaN,<NA>,None,125.3240,125.2186,0.1054,0.0437,0.0617,1,MACD (12h vs 26h EMA) is above signal line → b...
3,2024-05-01 03:00:00,126.1589,422406.4831,52954244.59,9489185,7529496,221.0711,0.000023,5.601521e+10,NaN,NaN,<NA>,None,125.4524,125.2882,0.1642,0.0678,0.0964,1,MACD (12h vs 26h EMA) is above signal line → b...
4,2024-05-01 04:00:00,125.9056,302809.5565,38198777.59,9238242,7418829,222.2989,0.000024,5.651554e+10,NaN,NaN,<NA>,None,125.5221,125.3339,0.1882,0.0919,0.0963,1,MACD (12h vs 26h EMA) is above signal line → b...


### Transaction Count Ratio

Compares the current hour's transaction count to the average of the past 24 hours.

It helps assess whether on-chain activity is surging, normal, or low.

In [45]:
# 1. Calculate 24h rolling average of transaction count
sol["txn_count_mean_24h"] = sol["transaction_count"].rolling(window=24).mean()

# 2. Compute the ratio: current transaction count / 24h rolling average
sol["txn_ratio"] = (sol["transaction_count"] / sol["txn_count_mean_24h"]).round(4)

# 3. Convert to transaction ratio description
sol["txn_ratio_description"] = sol["txn_ratio"].apply(
    lambda x: (
        "activity surge" if x > 1.1 else
        "low activity" if x < 0.9 else
        "normal activity"
    ) if pd.notna(x) else None
)

sol.drop(columns="txn_count_mean_24h", inplace=True)

In [46]:
sol["txn_ratio_description"].value_counts()

txn_ratio_description
normal activity    7225
activity surge      420
low activity        372
Name: count, dtype: int64

### Bollinger Bands

Bollinger Bands consist of three lines:
- Middle Band: Simple Moving Average (usually 20-period)
- Upper Band: Middle + 2 × standard deviation
- Lower Band: Middle - 2 × standard deviation

These bands help identify whether the price is relatively high or low
compared to recent history, and are useful for detecting breakout or sideways markets.

In [47]:
# 1. Calculate 20-period moving average (middle band)
sol["bb_mid"] = sol["close_price"].rolling(window=20).mean()

# 2. Calculate 20-period rolling standard deviation
sol["bb_std"] = sol["close_price"].rolling(window=20).std()

# 3. Compute upper and lower bands
sol["bb_upper"] = (sol["bb_mid"] + 2 * sol["bb_std"]).round(4)
sol["bb_lower"] = (sol["bb_mid"] - 2 * sol["bb_std"]).round(4)

# 4. Bollinger band width
sol["bb_position"] = sol.apply(
    lambda row: (
        "above upper band (possible breakout)" if row["close_price"] > row["bb_upper"]
        else "below lower band (possible breakdown)" if row["close_price"] < row["bb_lower"]
        else "within band (normal range)"
    ) if pd.notna(row["bb_upper"]) and pd.notna(row["bb_lower"]) else None, 
    axis=1
)

This field classifies the current close price relative to the Bollinger Bands.

- If the price is above the upper band → it's called a **breakout**.
    - Meaning: price has moved beyond normal volatility range on the upside.
    - Traders often interpret this as a strong bullish momentum or start of an upward trend.

- If the price is below the lower band → it's called a **rebound**.
    - Meaning: price may have dropped too sharply and is likely to bounce back.
    - Traders interpret this as a potential oversold signal or reversal opportunity.

- If the price is between upper and lower bands → it's in the **normal range** (sideways).
    - Meaning: market is moving within expected volatility bounds.

In [48]:
sol["bb_position"].value_counts()

bb_position
within band (normal range)               7123
below lower band (possible breakdown)     476
above upper band (possible breakout)      422
Name: count, dtype: int64

### Price Change Percentage

This indicator measures the relative change in closing price compared to the previous time step.

It's useful for identifying short-term price momentum (up/down/flat).

Formula:

   $\frac{\text{close}_t - \text{close}_{t-1}}{\text{close}_{t-1}}$

In [49]:
sol["price_change_pct"] = sol["close_price"].pct_change().round(6)
sol.head(10)

,timestamp,close_price,volume_base,volume_quote,transaction_count,successful_transaction_count,total_fee_sol,avg_fee_sol,market_cap,ma_6,...,macd_crossover_signal,macd_crossover_description,txn_ratio,txn_ratio_description,bb_mid,bb_std,bb_upper,bb_lower,bb_position,price_change_pct
0,2024-05-01 00:00:00,125.1005,5.139342e+05,6.457594e+07,8837436,6777203,228.6608,0.000026,5.706838e+10,NaN,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,NaN
1,2024-05-01 01:00:00,126.8711,6.834540e+05,8.616440e+07,9286854,7312422,206.7113,0.000022,5.602258e+10,NaN,...,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None,0.014153
2,2024-05-01 02:00:00,125.0548,7.058468e+05,8.831848e+07,8450646,6534633,217.1579,0.000026,5.677855e+10,NaN,...,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None,-0.014316
3,2024-05-01 03:00:00,126.1589,4.224065e+05,5.295424e+07,9489185,7529496,221.0711,0.000023,5.601521e+10,NaN,...,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None,0.008829
4,2024-05-01 04:00:00,125.9056,3.028096e+05,3.819878e+07,9238242,7418829,222.2989,0.000024,5.651554e+10,NaN,...,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None,-0.002008
5,2024-05-01 05:00:00,125.0820,3.206541e+05,4.016831e+07,9074138,7142826,171.0434,0.000019,5.628717e+10,125.6955,...,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None,-0.006541
6,2024-05-01 06:00:00,122.8263,6.899530e+05,8.534381e+07,8511218,6724752,162.6879,0.000019,5.588340e+10,125.3164,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.018034
7,2024-05-01 07:00:00,120.5031,1.757072e+06,2.122383e+08,8462130,6879615,202.1013,0.000024,5.527038e+10,124.2551,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.018915
8,2024-05-01 08:00:00,121.0997,1.108013e+06,1.329245e+08,8489463,6847295,182.6292,0.000022,5.386487e+10,123.5959,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,0.004951
9,2024-05-01 09:00:00,120.7857,7.428299e+05,9.014519e+07,8777861,7057795,179.6191,0.000020,5.375512e+10,122.7004,...,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None,-0.002593


In [50]:
# Classify movement for LLM prompt
sol["price_movement"] = sol["price_change_pct"].apply(
    lambda x: (
        "significant increase" if x > 0.01 else
        "slight increase" if x > 0.002 else
        "no significant change" if x > - 0.002 else
        "slight decrease" if x > -0.01 else
        "significant decrease"
    ) if pd.notna(x) else None 
)
sol["price_movement"].value_counts()

price_movement
slight increase          2364
slight decrease          2269
no significant change    1880
significant decrease      784
significant increase      742
Name: count, dtype: int64

In [51]:
sol.to_csv("SOL/sol_full.csv", index=False)

## DOGE

In [52]:
doge = pd.read_csv("DOGE/doge_final.csv", parse_dates=["timestamp"])
doge

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_doge,avg_fee_doge,total_size_used_bytes,market_cap
0,2024-05-01 00:00:00,0.131653,2.862902e+08,38105119.66,4731,6.674916e+08,0.230494,1593843,1.919231e+10
1,2024-05-01 01:00:00,0.131305,4.022567e+08,52859312.91,6419,3.206590e+08,0.184687,2186581,1.893531e+10
2,2024-05-01 02:00:00,0.130631,3.622108e+08,47070760.41,5364,3.654297e+08,0.251760,1925537,1.889457e+10
3,2024-05-01 03:00:00,0.132374,1.999534e+08,26345659.23,3761,3.075656e+08,0.394382,1889888,1.876737e+10
4,2024-05-01 04:00:00,0.132580,1.147719e+08,15225365.04,2124,2.631411e+08,0.514555,1459681,1.899158e+10
...,...,...,...,...,...,...,...,...,...
8035,2025-03-31 19:00:00,0.165746,1.546082e+08,25779667.50,1624,4.977779e+08,0.568891,2799532,2.484804e+10
8036,2025-03-31 20:00:00,0.165668,9.870469e+07,16412831.31,1772,8.277295e+08,0.433538,3357578,2.464346e+10
8037,2025-03-31 21:00:00,0.165791,5.603997e+07,9310295.35,1642,1.415250e+09,0.423078,3016915,2.464603e+10
8038,2025-03-31 22:00:00,0.165656,9.790216e+07,16160828.77,1354,6.266061e+08,0.466473,2494067,2.466526e+10


### Moving Average (MA)

In [53]:
doge["ma_6"] = doge["close_price"].rolling(window=6).mean().round(8)
doge["ma_24"] = doge["close_price"].rolling(window=24).mean().round(8)
doge["ma_crossover_signal"] = doge.apply(
    lambda row: (
        1 if row["ma_6"] > row["ma_24"]
        else 0
    ) if pd.notna(row["ma_6"]) and pd.notna(row["ma_24"]) else None,
    axis=1
).astype("Int64")    # 1: short MA is above long MA (buy signal), 0: short MA is below long MA (no buy signal)
doge["ma_crossover_description"] = doge["ma_crossover_signal"].apply(
    lambda x: (
        "short MA (6h) is above long MA (24h) → buy signal"
        if x == 1 else
        "short MA (6h) is below long MA (24h) → no buy signal"
        if x == 0 else 
        None
    ) if pd.notna(x) else None
)

### MACD (Moving Average Convergence Divergence)

`MACD`: a momentum indicator that shows the relationship between two Exponential Moving Averages (EMA)

`EMA`: gives more weight to recent prices, making it more responsive than a simple moving average (MA)

MACD consists of 3 main components:
1. MACD Line = EMA(12) - EMA(26): measures trend momentum.
2. Signal Line = EMA(9) of MACD: smooths the MACD line.
3. MACD Histogram = MACD Line - Signal Line: shows the strength of momentum change.

- A crossover (MACD > Signal) is commonly interpreted as a buy signal.
- A crossover (MACD < Signal) is commonly interpreted as a sell signal.

- 当 MACD线 > Signal线：表示上涨动量增强，通常是买入点
- 当 MACD线 < Signal线：表示下跌动量增强，通常是卖出点
- 两者的差值越大，说明动量越强


In [54]:
# 1. Compute short-term EMA (12h)
doge["ema_12"] = doge['close_price'].ewm(span=12, adjust=False).mean().round(8)

# 2. Compute long-term EMA (26h)
doge["ema_26"] = doge['close_price'].ewm(span=26, adjust=False).mean().round(8)

# 3. MACD line = ema_12 - ema_26
doge["macd"] = (doge["ema_12"] - doge["ema_26"]).round(8)

# 4. Signal line = EMA(9) of MACD3
    # 对 MACD 再做一次EMA（指数移动平均），它是一个“更平滑的 MACD 曲线”，用于生成买卖信号。
doge["macd_signal"] = doge["macd"].ewm(span=9, adjust=False).mean().round(8)

# 5. Compute MACD histogram (MACD - Signal)
doge["macd_hist"] = (doge["macd"] - doge["macd_signal"]).round(8)

# 6. Generate a crossover signal (1 -> MACD above signal -> buy momentum)
doge["macd_crossover_signal"] = (doge["macd"] > doge["macd_signal"]).astype(int)

# 7. Convert to MACD crossover description
doge["macd_crossover_description"] = doge["macd_crossover_signal"].apply(
    lambda x: (
        "MACD (12h vs 26h EMA) is above signal line → bullish momentum"
        if x == 1 else
        "MACD (12h vs 26h EMA) is below signal line → no bullish signal"
        if x == 0 else None
    )
)
doge.head()

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_doge,avg_fee_doge,total_size_used_bytes,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description
0,2024-05-01 00:00:00,0.131653,2.862902e+08,38105119.66,4731,6.674916e+08,0.230494,1593843,1.919231e+10,NaN,NaN,<NA>,None,0.131653,0.131653,0.000000,0.000000,0.000000,0,MACD (12h vs 26h EMA) is below signal line → n...
1,2024-05-01 01:00:00,0.131305,4.022567e+08,52859312.91,6419,3.206590e+08,0.184687,2186581,1.893531e+10,NaN,NaN,<NA>,None,0.131599,0.131627,-0.000028,-0.000006,-0.000022,0,MACD (12h vs 26h EMA) is below signal line → n...
2,2024-05-01 02:00:00,0.130631,3.622108e+08,47070760.41,5364,3.654297e+08,0.251760,1925537,1.889457e+10,NaN,NaN,<NA>,None,0.131450,0.131553,-0.000103,-0.000025,-0.000078,0,MACD (12h vs 26h EMA) is below signal line → n...
3,2024-05-01 03:00:00,0.132374,1.999534e+08,26345659.23,3761,3.075656e+08,0.394382,1889888,1.876737e+10,NaN,NaN,<NA>,None,0.131592,0.131614,-0.000022,-0.000024,0.000003,1,MACD (12h vs 26h EMA) is above signal line → b...
4,2024-05-01 04:00:00,0.132580,1.147719e+08,15225365.04,2124,2.631411e+08,0.514555,1459681,1.899158e+10,NaN,NaN,<NA>,None,0.131744,0.131685,0.000059,-0.000008,0.000067,1,MACD (12h vs 26h EMA) is above signal line → b...


### Transaction Count Ratio

Compares the current hour's transaction count to the average of the past 24 hours.

It helps assess whether on-chain activity is surging, normal, or low.

In [55]:
# 1. Calculate 24h rolling average of transaction count
doge["txn_count_mean_24h"] = doge["transaction_count"].rolling(window=24).mean()

# 2. Compute the ratio: current transaction count / 24h rolling average
doge["txn_ratio"] = (doge["transaction_count"] / doge["txn_count_mean_24h"]).round(4)

# 3. Convert to transaction ratio description
doge["txn_ratio_description"] = doge["txn_ratio"].apply(
    lambda x: (
        "activity surge" if x > 1.2 else
        "low activity" if x < 0.8 else
        "normal activity"
    ) if pd.notna(x) else None
)

doge.drop(columns="txn_count_mean_24h", inplace=True)

doge["txn_ratio_description"].value_counts()

txn_ratio_description
low activity       3390
normal activity    2771
activity surge     1856
Name: count, dtype: int64

### Bollinger Bands

Bollinger Bands consist of three lines:
- Middle Band: Simple Moving Average (usually 20-period)
- Upper Band: Middle + 2 × standard deviation
- Lower Band: Middle - 2 × standard deviation

These bands help identify whether the price is relatively high or low
compared to recent history, and are useful for detecting breakout or sideways markets.

In [56]:
# 1. Calculate 20-period moving average (middle band)
doge["bb_mid"] = doge["close_price"].rolling(window=20).mean()

# 2. Calculate 20-period rolling standard deviation
doge["bb_std"] = doge["close_price"].rolling(window=20).std()

# 3. Compute upper and lower bands
doge["bb_upper"] = (doge["bb_mid"] + 2 * doge["bb_std"]).round(8)
doge["bb_lower"] = (doge["bb_mid"] - 2 * doge["bb_std"]).round(8)

# 4. Bollinger band width
doge["bb_position"] = doge.apply(
    lambda row: (
        "above upper band (possible breakout)" if row["close_price"] > row["bb_upper"]
        else "below lower band (possible breakdown)" if row["close_price"] < row["bb_lower"]
        else "within band (normal range)"
    ) if pd.notna(row["bb_upper"]) and pd.notna(row["bb_lower"]) else None, 
    axis=1
)

This field classifies the current close price relative to the Bollinger Bands.

- If the price is above the upper band → it's called a **breakout**.
    - Meaning: price has moved beyond normal volatility range on the upside.
    - Traders often interpret this as a strong bullish momentum or start of an upward trend.

- If the price is below the lower band → it's called a **rebound**.
    - Meaning: price may have dropped too sharply and is likely to bounce back.
    - Traders interpret this as a potential oversold signal or reversal opportunity.

- If the price is between upper and lower bands → it's in the **normal range** (sideways).
    - Meaning: market is moving within expected volatility bounds.

In [57]:
doge.head(30)

,timestamp,close_price,volume_base,volume_quote,transaction_count,total_value_transferred_doge,avg_fee_doge,total_size_used_bytes,market_cap,ma_6,...,macd_hist,macd_crossover_signal,macd_crossover_description,txn_ratio,txn_ratio_description,bb_mid,bb_std,bb_upper,bb_lower,bb_position
0,2024-05-01 00:00:00,0.131653,2.862902e+08,3.810512e+07,4731,6.674916e+08,0.230494,1593843,1.919231e+10,NaN,...,0.000000,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
1,2024-05-01 01:00:00,0.131305,4.022567e+08,5.285931e+07,6419,3.206590e+08,0.184687,2186581,1.893531e+10,NaN,...,-0.000022,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
2,2024-05-01 02:00:00,0.130631,3.622108e+08,4.707076e+07,5364,3.654297e+08,0.251760,1925537,1.889457e+10,NaN,...,-0.000078,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
3,2024-05-01 03:00:00,0.132374,1.999534e+08,2.634566e+07,3761,3.075656e+08,0.394382,1889888,1.876737e+10,NaN,...,0.000003,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None
4,2024-05-01 04:00:00,0.132580,1.147719e+08,1.522537e+07,2124,2.631411e+08,0.514555,1459681,1.899158e+10,NaN,...,0.000067,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None
5,2024-05-01 05:00:00,0.131686,1.348075e+08,1.782979e+07,2649,3.423147e+08,0.373509,971719,1.904130e+10,0.131705,...,0.000046,1,MACD (12h vs 26h EMA) is above signal line → b...,NaN,None,NaN,NaN,NaN,NaN,None
6,2024-05-01 06:00:00,0.128946,2.326475e+08,3.032242e+07,8880,4.328481e+08,0.158072,2700905,1.894692e+10,0.131254,...,-0.000144,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
7,2024-05-01 07:00:00,0.124685,1.297384e+09,1.633415e+08,9649,5.019795e+08,0.179835,3236316,1.867317e+10,0.130150,...,-0.000527,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
8,2024-05-01 08:00:00,0.124329,9.614071e+08,1.187763e+08,9973,8.391744e+08,0.158568,3084028,1.794567e+10,0.129100,...,-0.000761,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None
9,2024-05-01 09:00:00,0.123978,3.560167e+08,4.440453e+07,29411,4.025960e+08,0.130700,8983604,1.775819e+10,0.127701,...,-0.000887,0,MACD (12h vs 26h EMA) is below signal line → n...,NaN,None,NaN,NaN,NaN,NaN,None


In [58]:
doge["bb_position"].value_counts()

bb_position
within band (normal range)               7119
below lower band (possible breakdown)     464
above upper band (possible breakout)      438
Name: count, dtype: int64

### Price Change Percentage

This indicator measures the relative change in closing price compared to the previous time step.

It's useful for identifying short-term price momentum (up/down/flat).

Formula:

   $\frac{\text{close}_t - \text{close}_{t-1}}{\text{close}_{t-1}}$

In [59]:
doge["price_change_pct"] = doge["close_price"].pct_change().round(6)
# Classify movement for LLM prompt
doge["price_movement"] = doge["price_change_pct"].apply(
    lambda x: (
        "significant increase" if x > 0.01 else
        "slight increase" if x > 0.002 else
        "no significant change" if x > - 0.002 else
        "slight decrease" if x > -0.01 else
        "significant decrease"
    ) if pd.notna(x) else None 
)
doge["price_movement"].value_counts()

price_movement
slight increase          2245
slight decrease          2127
no significant change    1843
significant decrease      928
significant increase      896
Name: count, dtype: int64

In [60]:
doge.to_csv("DOGE/doge_full.csv", index=False)

## USDT

In [61]:
usdt = pd.read_csv("USDT/usdt_final.csv", parse_dates=["timestamp"])
usdt

,timestamp,close_price,volume_base,volume_quote,market_cap
0,2024-05-01 00:00:00,0.999195,1.424318e+07,14231359.34,1.105692e+11
1,2024-05-01 01:00:00,0.999124,7.272293e+06,7267671.80,1.105374e+11
2,2024-05-01 02:00:00,0.999151,8.760352e+06,8755053.36,1.104664e+11
3,2024-05-01 03:00:00,0.999100,6.270335e+06,6266386.85,1.104993e+11
4,2024-05-01 04:00:00,0.999108,7.170144e+06,7166996.22,1.105020e+11
...,...,...,...,...,...
8035,2025-03-31 19:00:00,1.000039,2.419581e+07,24196501.77,1.439279e+11
8036,2025-03-31 20:00:00,1.000005,1.556304e+07,15561082.63,1.439114e+11
8037,2025-03-31 21:00:00,0.999940,2.478296e+07,24777574.63,1.439113e+11
8038,2025-03-31 22:00:00,0.999929,9.716986e+06,9716692.67,1.438995e+11


### Moving Average (MA)

In [62]:
usdt["ma_6"] = usdt["close_price"].rolling(window=6).mean().round(8)
usdt["ma_24"] = usdt["close_price"].rolling(window=24).mean().round(8)
usdt["ma_crossover_signal"] = usdt.apply(
    lambda row: (
        1 if row["ma_6"] > row["ma_24"]
        else 0
    ) if pd.notna(row["ma_6"]) and pd.notna(row["ma_24"]) else None,
    axis=1
).astype("Int64")    # 1: short MA is above long MA (buy signal), 0: short MA is below long MA (no buy signal)
usdt["ma_crossover_description"] = usdt["ma_crossover_signal"].apply(
    lambda x: (
        "short MA (6h) is above long MA (24h) → buy signal"
        if x == 1 else
        "short MA (6h) is below long MA (24h) → no buy signal"
        if x == 0 else 
        None
    ) if pd.notna(x) else None
)

### MACD (Moving Average Convergence Divergence)

`MACD`: a momentum indicator that shows the relationship between two Exponential Moving Averages (EMA)

`EMA`: gives more weight to recent prices, making it more responsive than a simple moving average (MA)

MACD consists of 3 main components:
1. MACD Line = EMA(12) - EMA(26): measures trend momentum.
2. Signal Line = EMA(9) of MACD: smooths the MACD line.
3. MACD Histogram = MACD Line - Signal Line: shows the strength of momentum change.

- A crossover (MACD > Signal) is commonly interpreted as a buy signal.
- A crossover (MACD < Signal) is commonly interpreted as a sell signal.

- 当 MACD线 > Signal线：表示上涨动量增强，通常是买入点
- 当 MACD线 < Signal线：表示下跌动量增强，通常是卖出点
- 两者的差值越大，说明动量越强


In [63]:
# 1. Compute short-term EMA (12h)
usdt["ema_12"] = usdt['close_price'].ewm(span=12, adjust=False).mean().round(8)

# 2. Compute long-term EMA (26h)
usdt["ema_26"] = usdt['close_price'].ewm(span=26, adjust=False).mean().round(8)

# 3. MACD line = ema_12 - ema_26
usdt["macd"] = (usdt["ema_12"] - usdt["ema_26"]).round(8)

# 4. Signal line = EMA(9) of MACD3
    # 对 MACD 再做一次EMA（指数移动平均），它是一个“更平滑的 MACD 曲线”，用于生成买卖信号。
usdt["macd_signal"] = usdt["macd"].ewm(span=9, adjust=False).mean().round(8)

# 5. Compute MACD histogram (MACD - Signal)
usdt["macd_hist"] = (usdt["macd"] - usdt["macd_signal"]).round(8)

# 6. Generate a crossover signal (1 -> MACD above signal -> buy momentum)
usdt["macd_crossover_signal"] = (usdt["macd"] > usdt["macd_signal"]).astype(int)

# 7. Convert to MACD crossover description
usdt["macd_crossover_description"] = usdt["macd_crossover_signal"].apply(
    lambda x: (
        "MACD (12h vs 26h EMA) is above signal line → bullish momentum"
        if x == 1 else
        "MACD (12h vs 26h EMA) is below signal line → no bullish signal"
        if x == 0 else None
    )
)
usdt.head()

,timestamp,close_price,volume_base,volume_quote,market_cap,ma_6,ma_24,ma_crossover_signal,ma_crossover_description,ema_12,ema_26,macd,macd_signal,macd_hist,macd_crossover_signal,macd_crossover_description
0,2024-05-01 00:00:00,0.999195,1.424318e+07,14231359.34,1.105692e+11,NaN,NaN,<NA>,None,0.999195,0.999195,0.000000,0.000000,0.000000,0,MACD (12h vs 26h EMA) is below signal line → n...
1,2024-05-01 01:00:00,0.999124,7.272293e+06,7267671.80,1.105374e+11,NaN,NaN,<NA>,None,0.999184,0.999189,-0.000006,-0.000001,-0.000004,0,MACD (12h vs 26h EMA) is below signal line → n...
2,2024-05-01 02:00:00,0.999151,8.760352e+06,8755053.36,1.104664e+11,NaN,NaN,<NA>,None,0.999179,0.999187,-0.000008,-0.000002,-0.000005,0,MACD (12h vs 26h EMA) is below signal line → n...
3,2024-05-01 03:00:00,0.999100,6.270335e+06,6266386.85,1.104993e+11,NaN,NaN,<NA>,None,0.999167,0.999180,-0.000013,-0.000005,-0.000009,0,MACD (12h vs 26h EMA) is below signal line → n...
4,2024-05-01 04:00:00,0.999108,7.170144e+06,7166996.22,1.105020e+11,NaN,NaN,<NA>,None,0.999158,0.999175,-0.000017,-0.000007,-0.000010,0,MACD (12h vs 26h EMA) is below signal line → n...


### Bollinger Bands

Bollinger Bands consist of three lines:
- Middle Band: Simple Moving Average (usually 20-period)
- Upper Band: Middle + 2 × standard deviation
- Lower Band: Middle - 2 × standard deviation

These bands help identify whether the price is relatively high or low
compared to recent history, and are useful for detecting breakout or sideways markets.

In [64]:
# 1. Calculate 20-period moving average (middle band)
usdt["bb_mid"] = usdt["close_price"].rolling(window=20).mean()

# 2. Calculate 20-period rolling standard deviation
usdt["bb_std"] = usdt["close_price"].rolling(window=20).std()

# 3. Compute upper and lower bands
usdt["bb_upper"] = (usdt["bb_mid"] + 2 * usdt["bb_std"]).round(8)
usdt["bb_lower"] = (usdt["bb_mid"] - 2 * usdt["bb_std"]).round(8)

# 4. Bollinger band width
usdt["bb_position"] = usdt.apply(
    lambda row: (
        "above upper band (possible breakout)" if row["close_price"] > row["bb_upper"]
        else "below lower band (possible breakdown)" if row["close_price"] < row["bb_lower"]
        else "within band (normal range)"
    ) if pd.notna(row["bb_upper"]) and pd.notna(row["bb_lower"]) else None, 
    axis=1
)

This field classifies the current close price relative to the Bollinger Bands.

- If the price is above the upper band → it's called a **breakout**.
    - Meaning: price has moved beyond normal volatility range on the upside.
    - Traders often interpret this as a strong bullish momentum or start of an upward trend.

- If the price is below the lower band → it's called a **rebound**.
    - Meaning: price may have dropped too sharply and is likely to bounce back.
    - Traders interpret this as a potential oversold signal or reversal opportunity.

- If the price is between upper and lower bands → it's in the **normal range** (sideways).
    - Meaning: market is moving within expected volatility bounds.

In [65]:
usdt["bb_position"].value_counts()

bb_position
within band (normal range)               7173
above upper band (possible breakout)      432
below lower band (possible breakdown)     416
Name: count, dtype: int64

In [66]:
usdt.to_csv("USDT/usdt_full.csv", index=False)